In [1]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

import cv2 as cv
import h5py
import lz4.frame
import matplotlib.dates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

import datetime
import pickle

import sys
import os
# Add the ptdraft folder path to the sys.path list
sys.path.append('/home/ryan/projects/Solar-Irradiance-Prediction-team07')
import utils.utils

np.set_printoptions(precision=4)

In [2]:
# hdf5_path = "/project/cq-training-1/project1/data/16bit-2014.01.01.0800.h5"
hdf5_path = "/home/ryan/data/16bit-2014.01.01.0800.h5"

hdf5_offset = 32  # this would correspond to: 2010.06.01.0800 + (32)*15min = 2010.06.01.1600
with h5py.File(hdf5_path, "r") as h5_data:
    print(list(h5_data.keys()))
    for data_set in enumerate(h5_data.items()):
        print(data_set)
    print(h5_data['BND_GHI'][:30])
    print(h5_data['BND_GHI_LUT'][:30])

['BND_GHI', 'BND_GHI_LUT', 'DRA_GHI', 'DRA_GHI_LUT', 'FPK_GHI', 'FPK_GHI_LUT', 'GWN_GHI', 'GWN_GHI_LUT', 'PSU_GHI', 'PSU_GHI_LUT', 'SXF_GHI', 'SXF_GHI_LUT', 'TBL_GHI', 'TBL_GHI_LUT', 'ch1', 'ch1_LUT', 'ch2', 'ch2_LUT', 'ch3', 'ch3_LUT', 'ch4', 'ch4_LUT', 'ch6', 'ch6_LUT', 'lat', 'lat_LUT', 'lon', 'lon_LUT', 'ncdf_path', 'ncdf_path_LUT']
(0, ('BND_GHI', <HDF5 dataset "BND_GHI": shape (96,), type "<f8">))
(1, ('BND_GHI_LUT', <HDF5 dataset "BND_GHI_LUT": shape (96,), type "<i4">))
(2, ('DRA_GHI', <HDF5 dataset "DRA_GHI": shape (96,), type "<f8">))
(3, ('DRA_GHI_LUT', <HDF5 dataset "DRA_GHI_LUT": shape (96,), type "<i4">))
(4, ('FPK_GHI', <HDF5 dataset "FPK_GHI": shape (96,), type "<f8">))
(5, ('FPK_GHI_LUT', <HDF5 dataset "FPK_GHI_LUT": shape (96,), type "<i4">))
(6, ('GWN_GHI', <HDF5 dataset "GWN_GHI": shape (96,), type "<f8">))
(7, ('GWN_GHI_LUT', <HDF5 dataset "GWN_GHI_LUT": shape (96,), type "<i4">))
(8, ('PSU_GHI', <HDF5 dataset "PSU_GHI": shape (96,), type "<f8">))
(9, ('PSU_GHI_LUT

In [3]:
target_channels = ["ch1", "ch2", "ch3", "ch4", "ch5"]
dataframe_path = "/home/ryan/data/catalog.helios.public.20100101-20160101.pkl"

In [4]:
df = pd.read_pickle(dataframe_path) if dataframe_path else None

In [5]:
print(*df.columns.to_list(), sep="\n")

ncdf_path
hdf5_8bit_path
hdf5_8bit_offset
hdf5_16bit_path
hdf5_16bit_offset
BND_DAYTIME
BND_CLEARSKY_GHI
BND_CLOUDINESS
BND_GHI
TBL_DAYTIME
TBL_CLEARSKY_GHI
TBL_CLOUDINESS
TBL_GHI
DRA_DAYTIME
DRA_CLEARSKY_GHI
DRA_CLOUDINESS
DRA_GHI
FPK_DAYTIME
FPK_CLEARSKY_GHI
FPK_CLOUDINESS
FPK_GHI
GWN_DAYTIME
GWN_CLEARSKY_GHI
GWN_CLOUDINESS
GWN_GHI
PSU_DAYTIME
PSU_CLEARSKY_GHI
PSU_CLOUDINESS
PSU_GHI
SXF_DAYTIME
SXF_CLEARSKY_GHI
SXF_CLOUDINESS
SXF_GHI


from datatime -> file -> offset -> get images -> samples -> batches....

In [6]:
import datetime
df[df.index == datetime.datetime.fromisoformat("2010-05-10 00:15:00")]

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,BND_CLOUDINESS,BND_GHI,TBL_DAYTIME,...,GWN_CLOUDINESS,GWN_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,PSU_CLOUDINESS,PSU_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI,SXF_CLOUDINESS,SXF_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,,,
2010-05-10 00:15:00,/project/cq-training-1/project1/data/netcdf/GO...,/project/cq-training-1/project1/data/hdf5v7_8b...,65,/project/cq-training-1/project1/data/hdf5v5_16...,65,1,33.066986,cloudy,81.106667,1,...,slightly cloudy,51.346667,1,0.0,cloudy,7.166667,1,146.092399,cloudy,55.9


# MLP on date and ghi

In [7]:
dataframe_path = "/home/ryan/data/catalog.helios.public.20100101-20160101.pkl"
dataframe = pd.read_pickle(dataframe_path) if dataframe_path else None
# dataframe = dataframe[dataframe.index >= datetime.datetime.fromisoformat('2015-01-01 08:00:00')]
# dataframe = dataframe[dataframe.index <= datetime.datetime.fromisoformat('2015-03-31 07:45:00')]

In [8]:
stations = {
      "BND": [40.05192, -88.37309, 230],
      "TBL": [40.12498, -105.23680, 1689],
      "DRA": [36.62373, -116.01947, 1007],
      "FPK": [48.30783, -105.10170, 634],
      "GWN": [34.25470, -89.87290, 98],
      "PSU": [40.72012, -77.93085, 376],
      "SXF": [43.73403, -96.62328, 473]
    }

In [9]:
# keep those columns intact
dataframe = dataframe.set_index(['ncdf_path', 'hdf5_8bit_path', 'hdf5_8bit_offset', 'hdf5_16bit_path', 'hdf5_16bit_offset'],
                  append=True)

# remove columns with about station not in the dictionary
stations_name = tuple(stations.keys())
dataframe = dataframe.loc[:, dataframe.columns.str.startswith(stations_name)]

# create column for station name and change other columns to not be station specific
# (e.g. replace columns 'BND_GHI', TBL_GHI','DRA_GHI', etc by columns STATION_NAME and GHI)
# each rows contain information about only one station
dataframe.columns = dataframe.columns.str.split('_', 1, expand=True)
dataframe = dataframe.stack(0).reset_index().rename(columns={'level_6': 'STATION_NAME'}).set_index(['iso-datetime'])

# add column station index and add it as index (with the datetime)
dataframe['STATION_INDEX'] = dataframe.apply(lambda row: stations_name.index(row['STATION_NAME']), axis=1)
dataframe = dataframe.set_index(['STATION_INDEX'],append=True)

# sort dataframe so that the index correspond between the dataframe and the images np.array

dataframe.sort_index( inplace=True)

In [10]:
dataframe=dataframe.reset_index()

In [11]:
dataframe.head(10)

,iso-datetime,STATION_INDEX,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,STATION_NAME,CLEARSKY_GHI,CLOUDINESS,DAYTIME,GHI
0,2010-01-01 00:00:00,0,nan,nan,NaN,nan,NaN,BND,0.000000,night,0,-4.100000
1,2010-01-01 00:00:00,1,nan,nan,NaN,nan,NaN,TBL,0.000000,night,0,-3.600000
2,2010-01-01 00:00:00,2,nan,nan,NaN,nan,NaN,DRA,52.706473,cloudy,1,NaN
3,2010-01-01 00:00:00,3,nan,nan,NaN,nan,NaN,FPK,0.000000,night,0,-3.100000
4,2010-01-01 00:00:00,4,nan,nan,NaN,nan,NaN,GWN,0.000000,night,0,-7.500000
5,2010-01-01 00:00:00,5,nan,nan,NaN,nan,NaN,PSU,0.000000,night,0,-1.500000
6,2010-01-01 00:00:00,6,nan,nan,NaN,nan,NaN,SXF,0.000000,night,0,-3.800000
7,2010-01-01 00:15:00,0,nan,nan,NaN,nan,NaN,BND,0.000000,night,0,-4.100000
8,2010-01-01 00:15:00,1,nan,nan,NaN,nan,NaN,TBL,0.000000,night,0,-5.353333
9,2010-01-01 00:15:00,2,nan,nan,NaN,nan,NaN,DRA,18.742351,cloudy,1,NaN


In [12]:
# station_idx = 0
# iso_datetime = "2014-11-20 23:30:00"
# def get_one_sample(dataframe, station_idx, iso_datetime):
#     if station_idx is not None:
#         n = dataframe.index.get_loc((iso_datetime, station_idx))
#     else:
#         n = dataframe.index.get_loc((iso_datetime))
# #     print(dataframe.iloc[n])
#     return dataframe.iloc[n]
    
# sample = get_one_sample(dataframe, station_idx, iso_datetime)   

In [13]:
# print(sample.index)

In [14]:
from timezonefinder import TimezoneFinder
import pytz

station_timezone = []
for station_name in stations.keys():
    tf = TimezoneFinder()
#     latitude, longitude = 52.5061, 13.358
    latitude, longitude = stations[station_name][0], stations[station_name][1]
    timezone = tf.timezone_at(lng=longitude, lat=latitude) # returns 'Europe/Berlin'
    print(timezone)
    station_timezone.append(pytz.timezone(timezone))

America/Chicago
America/Denver
America/Los_Angeles
America/Denver
America/Chicago
America/New_York
America/Chicago


In [15]:
# from datetime import datetime, timedelta
# from pytz import timezone
import datetime
import pytz

iso_datetime = "2014-11-20 23:30:00"

fmt= "%Y-%m-%d %H:%M:%S"
dayofyear = "%j"
utc_time = datetime.datetime.strptime(iso_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc)

station_local_time = []
for tz in station_timezone:
    loc_dt = utc_time.astimezone(tz)
    print(loc_dt.strftime(fmt)) 
    print(loc_dt.strftime("%j"))
    station_local_time.append(loc_dt)

2014-11-20 17:30:00
324
2014-11-20 16:30:00
324
2014-11-20 15:30:00
324
2014-11-20 16:30:00
324
2014-11-20 17:30:00
324
2014-11-20 18:30:00
324
2014-11-20 17:30:00
324


In [16]:
unidata = dataframe[['iso-datetime', 'STATION_INDEX', 'CLOUDINESS', 'DAYTIME','GHI']]

In [17]:
import datetime
def convert_time(timestamp):
    """
    Take the hour/minute and month of the timestamp and convert them to a 
    sin/cos vector to better represent the similarity between January (1)
    and December (12) or 23h45 and 00h00
    This function can take a string (json file) or a timestamp object (pd dataframe)
    """
    try:
        date = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    except TypeError:
        date = timestamp
    min15_in_day = 24*4
    sin_month = np.sin(2*np.pi*date.month/12)
    cos_month = np.cos(2*np.pi*date.month/12)
    sin_minute = np.sin(2*np.pi*(date.hour*4+date.minute/15)/min15_in_day)
    cos_minute = np.cos(2*np.pi*(date.hour*4+date.minute/15)/min15_in_day)
    return (sin_month,cos_month,sin_minute,cos_minute)

print(convert_time("2015-01-01 02:00:00"))

(0.49999999999999994, 0.8660254037844387, 0.49999999999999994, 0.8660254037844387)


In [18]:
unidata.head(10)

# https://stackoverflow.com/questions/34279378/python-pandas-apply-function-with-two-arguments-to-columns/34279543

def get_local_time(input_vector):
    iso_datetime = str(input_vector[0])
    station_idx = input_vector[1]
    fmt= "%Y-%m-%d %H:%M:%S"
    dayofyear = "%-j"
    utc_time = datetime.datetime.strptime(iso_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc)
    
    tz= station_timezone[station_idx]
    loc_dt = utc_time.astimezone(tz)
#     return loc_dt.strftime(fmt)
    sin_month,cos_month,sin_minute,cos_minute = convert_time(loc_dt.strftime(fmt))
    return pd.Series(convert_time(loc_dt.strftime(fmt)))


# unidata['localtime'] = unidata[['iso-datetime', 'STATION_INDEX']].apply(get_local_time, axis=1)# 
unidata[['sin_month','cos_month','sin_minute','cos_minute']] = unidata[['iso-datetime', 'STATION_INDEX']].apply(get_local_time, axis=1)

/home/ryan/projects/Solar-Irradiance-Prediction-team07/solar_env/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
# https://stackoverflow.com/questions/38088652/pandas-convert-categories-to-numbers

# unidata.cc = pd.Categorical(df.cc)
# unidata['code'] = df.cc.cat.codes
unidata['CLOUDINESS_'] = unidata.CLOUDINESS.astype('category').cat.codes

/home/ryan/projects/Solar-Irradiance-Prediction-team07/solar_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
unidata.head(20)

,iso-datetime,STATION_INDEX,CLOUDINESS,DAYTIME,GHI,sin_month,cos_month,sin_minute,cos_minute,CLOUDINESS_
0,2010-01-01 00:00:00,0,night,0,-4.100000,-2.449294e-16,1.0,-1.000000,-1.836970e-16,2
1,2010-01-01 00:00:00,1,night,0,-3.600000,-2.449294e-16,1.0,-0.965926,-2.588190e-01,2
2,2010-01-01 00:00:00,2,cloudy,1,NaN,-2.449294e-16,1.0,-0.866025,-5.000000e-01,1
3,2010-01-01 00:00:00,3,night,0,-3.100000,-2.449294e-16,1.0,-0.965926,-2.588190e-01,2
4,2010-01-01 00:00:00,4,night,0,-7.500000,-2.449294e-16,1.0,-1.000000,-1.836970e-16,2
5,2010-01-01 00:00:00,5,night,0,-1.500000,-2.449294e-16,1.0,-0.965926,2.588190e-01,2
6,2010-01-01 00:00:00,6,night,0,-3.800000,-2.449294e-16,1.0,-1.000000,-1.836970e-16,2
7,2010-01-01 00:15:00,0,night,0,-4.100000,-2.449294e-16,1.0,-0.997859,6.540313e-02,2
8,2010-01-01 00:15:00,1,night,0,-5.353333,-2.449294e-16,1.0,-0.980785,-1.950903e-01,2
9,2010-01-01 00:15:00,2,cloudy,1,NaN,-2.449294e-16,1.0,-0.896873,-4.422887e-01,1


In [21]:
unidata = unidata.dropna()

In [22]:
unidata.to_csv("unidata.csv")

In [23]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
train,test = train_test_split(unidata, test_size=0.20)

In [51]:
# X = train[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute']]
X = train[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute','cos_month','cos_minute']]

# X = train[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute', 'CLOUDINESS_']]

X = X.values.tolist()
y = train["GHI"]
y = y.values.tolist()
regressor_8_8 = MLPRegressor(solver='adam', alpha=1e-5, hidden_layer_sizes=(8, 8), random_state=1, max_iter = 500, early_stopping=True, verbose=True)
regressor_8_8.fit(X, y)
# pickle.dump(regressor_8_8, open(f'mlp_regrssor_8_8.sav', 'wb'))

Iteration 1, loss = 15826.16546099
Validation score: 0.774862
Iteration 2, loss = 8420.51018970
Validation score: 0.782690
Iteration 3, loss = 8191.29414192
Validation score: 0.787115
Iteration 4, loss = 8066.46401319
Validation score: 0.789893
Iteration 5, loss = 7991.94515024
Validation score: 0.791444
Iteration 6, loss = 7941.30254264
Validation score: 0.792703
Iteration 7, loss = 7898.13184043
Validation score: 0.794042
Iteration 8, loss = 7837.96225215
Validation score: 0.795218
Iteration 9, loss = 7795.58226139
Validation score: 0.796217
Iteration 10, loss = 7770.22973330
Validation score: 0.796777
Iteration 11, loss = 7753.75545071
Validation score: 0.796951
Iteration 12, loss = 7742.81527647
Validation score: 0.797256
Iteration 13, loss = 7732.02443753
Validation score: 0.797611
Iteration 14, loss = 7722.44801130
Validation score: 0.797874
Iteration 15, loss = 7712.20470779
Validation score: 0.798000
Iteration 16, loss = 7703.25328138
Validation score: 0.798329
Iteration 17, lo

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(8, 8), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [52]:
from sklearn.metrics import r2_score,mean_squared_error

def evaluate(model):
    # X_test = test[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute']]
    X_test = test[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute','cos_month','cos_minute']]
    # X_test = test[['STATION_INDEX', 'DAYTIME','sin_month','sin_minute', 'CLOUDINESS_']]
    X_test = X_test.values.tolist()
    y_test = test["GHI"]
    y_test = y_test.values.tolist()

    y_predict = model.predict(X_test)
    print("RMSE= %.3f" % np.sqrt(mean_squared_error(y_test, y_predict)))

    print("R2_score= %.3f" % r2_score(y_test, y_predict))
    for label, predict in zip(y_test[:10], y_predict[:10]):
        print("delta: %0.3f,\t label: %0.3f, prediction: %0.3f" % (label- predict, label, predict))
    

In [53]:
evaluate(regressor_8_8)

RMSE= 122.186
R2_score= 0.805
delta: 0.883,	 label: -3.833, prediction: -4.716
delta: -482.678,	 label: 175.727, prediction: 658.405
delta: 300.520,	 label: 672.747, prediction: 372.226
delta: 3.332,	 label: -1.567, prediction: -4.899
delta: 4.308,	 label: 1.620, prediction: -2.688
delta: 194.316,	 label: 336.187, prediction: 141.871
delta: 1.412,	 label: -3.153, prediction: -4.566
delta: -345.814,	 label: 391.013, prediction: 736.827
delta: 100.517,	 label: 409.507, prediction: 308.990
delta: 0.474,	 label: -4.533, prediction: -5.007


In [54]:
regressor_32_16 = MLPRegressor(activation='relu', solver='adam', alpha=1e-5, learning_rate='adaptive', hidden_layer_sizes=(32,16), random_state=1, max_iter = 500, early_stopping=True, verbose=True)

regressor_32_16.fit(X, y)
evaluate(regressor_32_16)
pickle.dump(regressor_32_16, open(f'mlp_regressor_32_16.sav', 'wb'))

Iteration 1, loss = 11554.76719643
Validation score: 0.792936
Iteration 2, loss = 7917.90950714
Validation score: 0.798060
Iteration 3, loss = 7772.29329586
Validation score: 0.800826
Iteration 4, loss = 7673.77598002
Validation score: 0.803627
Iteration 5, loss = 7605.00785827
Validation score: 0.804971
Iteration 6, loss = 7551.58240860
Validation score: 0.806475
Iteration 7, loss = 7509.14563671
Validation score: 0.806727
Iteration 8, loss = 7476.63378162
Validation score: 0.807428
Iteration 9, loss = 7453.26165861
Validation score: 0.808188
Iteration 10, loss = 7438.90308756
Validation score: 0.809004
Iteration 11, loss = 7429.42551895
Validation score: 0.809120
Iteration 12, loss = 7420.18311781
Validation score: 0.808583
Iteration 13, loss = 7410.30361312
Validation score: 0.809422
Iteration 14, loss = 7393.41096818
Validation score: 0.810216
Iteration 15, loss = 7385.42906318
Validation score: 0.810331
Iteration 16, loss = 7379.06469185
Validation score: 0.810370
Iteration 17, lo

In [50]:
evaluate(regressor_32_16)

RMSE= 61.140
0.951228728194173
delta: -0.522,	 label: -3.833, prediction: -3.311
delta: -77.075,	 label: 175.727, prediction: 252.801
delta: 129.060,	 label: 672.747, prediction: 543.686
delta: 1.992,	 label: -1.567, prediction: -3.559
delta: 2.271,	 label: 1.620, prediction: -0.651
delta: 126.284,	 label: 336.187, prediction: 209.902
delta: 1.959,	 label: -3.153, prediction: -5.113
delta: 83.257,	 label: 391.013, prediction: 307.756
delta: 5.351,	 label: 409.507, prediction: 404.156
delta: 0.617,	 label: -4.533, prediction: -5.150


In [28]:
from timezonefinder import TimezoneFinder

def convert_utc_localtime(stations, station_idx, iso_datetime):
    fmt= "%Y-%m-%d %H:%M:%S"
    utc_time = datetime.strptime(iso_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc)
    
    station_timezone = []
    for station_name in stations.keys():
        tf = TimezoneFinder()
    #     latitude, longitude = 52.5061, 13.358
        latitude, longitude = stations[station_name][0], stations[station_name][1]
        timezone = tf.timezone_at(lng=longitude, lat=latitude)
        print(timezone)
        station_timezone.append(pytz.timezone(timezone))
        
    tz= station_timezone[station_idx]
    loc_dt = utc_time.astimezone(tz)
    return loc_dt.strftime(fmt)

In [31]:
len(X)

1173013